In [ ]:
import polars as pl
df = safe_fillna_polars(df)  # 或者用 pandas 的 safe_fillna


In [ ]:
import polars as pl
df = safe_fillna_polars(df)  # 或者用 pandas 的 safe_fillna


In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

X = torch.tensor(X_scaled, dtype=torch.float32)

# 随机遮挡 15% 的因子
mask_ratio = 0.15
mask = torch.rand_like(X) < mask_ratio
X_masked = X.clone()
X_masked[mask] = 0.0  # 0表示mask

dataset = TensorDataset(X_masked, X)  # 输入 = mask 后, 目标 = 原始因子
loader = DataLoader(dataset, batch_size=64, shuffle=True)

# 简单 Transformer
class FactorTransformer(nn.Module):
    def __init__(self, n_features, d_model=64, n_heads=4, n_layers=2):
        super().__init__()
        self.embedding = nn.Linear(n_features, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=n_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)
        self.decoder = nn.Linear(d_model, n_features)

    def forward(self, x):
        x = self.embedding(x)  # [batch, n_features] -> [batch, d_model]
        x = x.unsqueeze(1)     # [batch, seq=1, d_model]
        x = self.transformer(x).squeeze(1)
        return self.decoder(x)

model = FactorTransformer(n_features=X.shape[1])
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

# 训练
for epoch in range(20):
    for xb, yb in loader:
        optimizer.zero_grad()
        pred = model(xb)
        loss = loss_fn(pred, yb)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} loss={loss.item():.6f}")


NameError: name 'X_scaled' is not defined

In [ ]:
embeddings = model.embedding(X)  # [batch, d_model]

# 下游预测未来收益率
from sklearn.linear_model import LinearRegression
y = torch.tensor(future_returns, dtype=torch.float32)
reg = LinearRegression().fit(embeddings.detach().numpy(), y.numpy())


In [ ]:
from pytorch_tabnet.tab_model import TabNetRegressor
tabnet = TabNetRegressor()
tabnet.fit(X_train=embeddings.detach().numpy(), y_train=y.numpy())
